In [ ]:
from huggingface_hub import login
login("API token here")   # Paste your Hugging Face API token here

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
from sklearn.metrics import classification_report
import torch
from torch.utils.data import DataLoader

# Load the SST-2 dataset
dataset = load_dataset("glue", "sst2")

# Load the pre-trained Llama model and tokenizer
model_name = "meta-llama/Llama-3.2-1B" 
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Explicitly set the padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Tokenize the dataset
def preprocess_function(examples):
    return tokenizer(examples["sentence"], truncation=True, padding=True, max_length=128)

encoded_dataset = dataset.map(preprocess_function, batched=True)
test_dataset = encoded_dataset["validation"]  # Use validation split for testing

# Convert dataset to PyTorch format
test_dataset = test_dataset.remove_columns(["sentence", "idx"])  # Remove unnecessary columns
test_dataset.set_format("torch")  # Ensure the data is in PyTorch format

test_loader = DataLoader(test_dataset, batch_size=1)

# Evaluate the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

true_labels = []
predicted_labels = []

with torch.no_grad():
    for batch in test_loader:
        inputs = {key: batch[key].to(device) for key in ["input_ids", "attention_mask"]}
        labels = batch["label"].to(device)

        outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=-1)

        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(predictions.cpu().numpy())

# Compute metrics
report = classification_report(true_labels, predicted_labels, target_names=["negative", "positive"], digits=4)
print("Classification Metrics:")
print(report)


README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

...

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-3.2-1B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

Classification Metrics:
              precision    recall  f1-score   support

    negative     0.4902    0.9977    0.6574       428
    positive     0.0000    0.0000    0.0000       444

    accuracy                         0.4897       872
   macro avg     0.2451    0.4988    0.3287       872
weighted avg     0.2406    0.4897    0.3227       872

